##Extraction de texte_Deroulement de vol

In [179]:
!pip install fitz

In [180]:
!pip install PyMuPDF

In [181]:
!pip install pandas

In [182]:
import fitz  # PyMuPDF
import os
import pandas as pd
import re

# Dossier contenant les fichiers PDF
dossier_pdf = "/content/drive/MyDrive/TPE_Mon_Travail/Rapport_BEA/12DM.pdf"

# Expression régulière pour les sous-titres de début et de fin
sous_titre_debut_pattern = re.compile(r"DÉROULEMENT DU VOL")
sous_titre_fin_pattern = re.compile(r"RENSEIGNEMENTS COMPLÉMENTAIRES")

# Initialisez une liste pour stocker les données extraites
donnees_extraites = []

# Ouvrir le fichier PDF
doc = fitz.open(dossier_pdf)

# Initialisez le texte extrait pour ce fichier
texte_extrait = ""

# Parcourir chaque page du PDF
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    page_text = page.get_text()

    # Utilisez des expressions régulières pour trouver les sous-titres de début et de fin
    debut_match = sous_titre_debut_pattern.search(page_text)
    fin_match = sous_titre_fin_pattern.search(page_text)

    # Si l'un des sous-titres est trouvé, extrayez le texte
    if debut_match or fin_match:
        if debut_match:
            debut = debut_match.end()
        else:
            debut = 0  # Commence depuis le début de la page

        if fin_match:
            fin = fin_match.start()
        else:
            fin = len(page_text)  # Prend toute la fin de la page

        # Récupérez le texte entre les deux sous-titres ou jusqu'à la fin de la page
        texte_entre_sous_titres = page_text[debut:fin].strip()
        texte_extrait += texte_entre_sous_titres + "\n"

# Ajoutez les données extraites à la liste
donnees_extraites.append(("12DM.pdf", texte_extrait))

# Fermez le fichier PDF
doc.close()

# Créez un DataFrame avec les données extraites et ajoutez une colonne "id"
df = pd.DataFrame(donnees_extraites, columns=["id", "Deroulement"])

# Enregistrez les données extraites dans un fichier CSV
df.to_csv("/content/drive/MyDrive/TPE_Mon_Travail/Déroulement_Vol_Test.csv", index=True)

# Affichez les données extraites en colonnes 'id' et 'Deroulement'
print(df[['id', 'Deroulement']])

# Affichez le message de confirmation
print("Données extraites et enregistrées avec succès.")


         id                                        Deroulement
0  12DM.pdf  Note : Les informations suivantes sont princip...
Données extraites et enregistrées avec succès.


##Nettoyage de l'extraction du deroulement du vol

In [183]:
import pandas as pd
import re
from google.colab import drive

# Monter Google Drive pour accéder aux fichiers
drive.mount('/content/drive')

# Chemin vers le fichier CSV sur Google Drive
csv_file_path = "/content/drive/MyDrive/TPE_Mon_Travail/Déroulement_Vol_Test.csv"

# Lire le fichier CSV en utilisant pandas
data = pd.read_csv(csv_file_path, encoding='utf-8')

# Supprimer l'expression "Conséquences et dommages" de la colonne "contenu"
#data['contenu'] = data['contenu'].str.replace('Conséquences et dommages', '')

# Liste des catégories d'éléments à supprimer
categories_a_supprimer = {
    "Articles définis": ["le", "la", "les", "l'"],
    "Prépositions et articles partitifs": ["de", "du", "des", "d'", "d", "au", "aux", "en"],
    "Articles indéfinis": ["un", "une"],
    "Prépositions": ["à", "dans", "sur", "pour", "par", "pas", "sans", "avec", "où"],
    "Conjonction de coordination": ["et", "car", "ou"],
    "Pronoms relatifs": ["que", "qui", "qu'"],
    "Pronoms personnels et réflexifs": ["il", "nous", "ce", "ces", "se", "ceci", "donc", "dont", "alors", "lors", "autre","en"]
}

# Fonction pour nettoyer le texte en supprimant les articles et les mots courts
def nettoyer_texte(texte, mots_a_supprimer):
    # Utiliser une expression régulière pour séparer les mots
    mots = re.findall(r'\b\w+\b', str(texte))

    # Filtrer les mots en supprimant les articles, les mots spécifiques
    # et les mots courts (un ou deux caractères)
    mots_filtrés = [mot for mot in mots if len(mot) > 2 and mot.lower() not in mots_a_supprimer]

    # Rejoindre les mots filtrés pour former le texte nettoyé
    texte_nettoyé = ' '.join(mots_filtrés)
    return texte_nettoyé

# Appliquer la fonction de nettoyage à la colonne de contenu
for categorie, mots_a_supprimer in categories_a_supprimer.items():
    data['Deroulement'] = data['Deroulement'].apply(lambda x: nettoyer_texte(x, mots_a_supprimer))

# Supprimer la colonne "Unnamed: 0"
data = data.drop(columns=['Unnamed: 0'])

# Supprimer le chemin "/content/drive/MyDrive/TPE_Mon_Travail/Rapport_BEA" de la colonne 'id'
#data['id'] = data['id'].str.split("/").str[-1]

# Enregistrer le résultat dans un nouveau fichier CSV sur Google Drive avec un encodage spécifique (par exemple, UTF-8)
cleaned_csv_file_path = "/content/drive/MyDrive/TPE_Mon_Travail/Déroul_Vol_Test_Nett.csv"
data.to_csv(cleaned_csv_file_path, index=False, encoding='utf-8')

# Afficher les premières lignes du DataFrame nettoyé
print(data)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
         id                                        Deroulement
0  12DM.pdf  Note informations suivantes sont principalemen...


##Appliquons le TF_IDF sur l'extraction du Déroul_Vol_Test_Nett

In [184]:
!pip install pandas

In [185]:
!pip install numpy

In [186]:
!pip install pandas scikit-learn

In [187]:
!pip install nltk>=3.6

In [188]:
pip install ngrams

##Le TF_IDF sur le contenu_Deroulement du vol

In [189]:
import pandas as pd
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer

# Charger le DataFrame
input_path = "/content/drive/MyDrive/TPE_Mon_Travail/Déroul_Vol_Test_Nett.csv"
data = pd.read_csv(input_path)

# Tokenization : Divisez le texte en n-grammes
data['tokens'] = data['Deroulement'].apply(lambda x: list(ngrams(x.lower().split(), 2)))  # Changer 2 ici pour la longueur de vos n-grammes

# Configuration de TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Changer la plage ngram ici si nécessaire

# Entraînement de TF-IDF
X = vectorizer.fit_transform([" ".join(map(str, doc)) for doc in data['tokens']])

# Obtenir les vecteurs de documents
document_vectors = X.toarray()

# Créez un DataFrame pour les vecteurs de documents
vectors_df = pd.DataFrame(document_vectors, columns=[f"d_{i}" for i in range(X.shape[1])])

# Créer un DataFrame avec la colonne "id"
id_df = data[['id']]

# Enregistrez le DataFrame résultant
output_path = "/content/drive/MyDrive/TPE_Mon_Travail/Déroul_Vol_Test_Nett_TF-IDF.csv"
result_df = pd.concat([id_df, vectors_df], axis=1)
result_df.to_csv(output_path, index=True)

# Affichez le DataFrame résultant avec la colonne "id" et les vecteurs de documents
print(result_df)


         id       d_0       d_1       d_2       d_3       d_4       d_5  \
0  12DM.pdf  0.071112  0.035556  0.035556  0.071112  0.035556  0.035556   

        d_6       d_7       d_8  ...     d_293     d_294     d_295     d_296  \
0  0.071112  0.035556  0.035556  ...  0.106668  0.071112  0.035556  0.035556   

      d_297     d_298     d_299     d_300     d_301     d_302  
0  0.071112  0.035556  0.035556  0.071112  0.035556  0.035556  

[1 rows x 304 columns]


## Fusion Bow_Tilte and TF_IDF

In [190]:
import pandas as pd

# Chemins d'accès vers les fichiers CSV
path_bow = "/content/drive/MyDrive/TPE_Mon_Travail/BD1-Cluster3_Title.csv"
path_tfidf = "/content/drive/MyDrive/TPE_Mon_Travail/Déroul_Vol_Nett_TF-IDF.csv"

# Charger les deux DataFrames
df_bow = pd.read_csv(path_bow)
df_tfidf = pd.read_csv(path_tfidf)

# Fusionner les deux DataFrames en fonction de la colonne "id"
merged_df = pd.merge(df_bow, df_tfidf, on="id")

# Déplacer la colonne "cluster" à la fin du DataFrame fusionné
cluster_column = merged_df.pop("cluster")
merged_df["cluster"] = cluster_column

# Enregistrer le DataFrame fusionné dans un nouveau fichier CSV
output_path = "/content/drive/MyDrive/TPE_Mon_Travail/BD_Fusion-TF_IDF_D-Vol.csv"
merged_df.to_csv(output_path, index=True)

# Afficher le DataFrame fusionné
print(merged_df)


                   id  agent  arrière  autogir  avion  avoir  aéronef  \
0          F-HPTI.pdf      0        0        0      0      0        0   
1           21AJD.pdf      0        0        0      1      0        0   
2          F-GTPV.pdf      0        0        0      1      0        0   
3          F-HMYY.pdf      0        0        0      0      0        0   
4          F-GYFR.pdf      0        0        0      0      0        0   
..                ...    ...      ...      ...    ...    ...      ...   
195  BEA2020-0165.pdf      0        0        0      1      0        0   
196  BEA2020-0418.pdf      0        0        0      0      0        0   
197  BEA2019-0289.pdf      0        0        0      1      0        0   
198  BEA2020-0373.pdf      0        0        0      2      0        0   
199  BEA2020-0094.pdf      0        0        0      1      0        0   

     balisage  ballon  blesser  ...  d_1991    d_1992    d_1993  d_1994  \
0           0       0        0  ...     0.0  0.0

In [191]:
df_bow = pd.read_csv(path_bow)
path_tfidf = pd.read_csv(path_tfidf)
df_cluster=df_bow.drop(columns=df_bow.columns[1:-1])
merged_df = df_cluster.merge(df_tfidf, on="id")

# Déplacer la colonne "cluster" à la fin du DataFrame fusionné
cluster_column = merged_df.pop("cluster")
merged_df["cluster"] = cluster_column

# Supprimer la colonne "Unnamed: 0"
merged_df = merged_df.drop(columns=["Unnamed: 0"])

# Enregistrer le DataFrame fusionné dans un nouveau fichier CSV
output_path = "/content/drive/MyDrive/TPE_Mon_Travail/BD_Fusion-TF_IDF_D-Vol1.csv"
merged_df.to_csv(output_path, index=True)

# Afficher le DataFrame fusionné
print(merged_df)


                   id       d_0       d_1  d_2       d_3       d_4  d_5  d_6  \
0          F-HPTI.pdf  0.000000  0.000000  0.0  0.000000  0.000000  0.0  0.0   
1           21AJD.pdf  0.000000  0.000000  0.0  0.000000  0.000000  0.0  0.0   
2          F-GTPV.pdf  0.000000  0.000000  0.0  0.000000  0.000000  0.0  0.0   
3          F-HMYY.pdf  0.052311  0.026155  0.0  0.000000  0.000000  0.0  0.0   
4          F-GYFR.pdf  0.000000  0.000000  0.0  0.000000  0.000000  0.0  0.0   
..                ...       ...       ...  ...       ...       ...  ...  ...   
195  BEA2020-0165.pdf  0.000000  0.000000  0.0  0.000000  0.000000  0.0  0.0   
196  BEA2020-0418.pdf  0.046865  0.023432  0.0  0.000000  0.000000  0.0  0.0   
197  BEA2019-0289.pdf  0.000000  0.000000  0.0  0.068323  0.034162  0.0  0.0   
198  BEA2020-0373.pdf  0.000000  0.000000  0.0  0.000000  0.000000  0.0  0.0   
199  BEA2020-0094.pdf  0.000000  0.000000  0.0  0.000000  0.000000  0.0  0.0   

     d_7  d_8  ...  d_1991    d_1992   

In [192]:
df_tfidf

,Unnamed: 0,id,d_0,d_1,d_2,d_3,d_4,d_5,d_6,d_7,...,d_1990,d_1991,d_1992,d_1993,d_1994,d_1995,d_1996,d_1997,d_1998,d_1999
0,0,F-HPTI.pdf,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,1,21AJD.pdf,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,2,F-GTPV.pdf,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,3,F-HMYY.pdf,0.052311,0.026155,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,4,F-GYFR.pdf,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,196,BEA2020-0418.pdf,0.046865,0.023432,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
197,197,BEA2019-0289.pdf,0.000000,0.000000,0.0,0.068323,0.034162,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.087591,0.043796,0.0,0.0
198,198,BEA2020-0373.pdf,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
199,199,BEA2020-0094.pdf,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


##Similarite

In [193]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Chemin d'accès du document de test (déjà TF-IDF)
path_test_document = "/content/drive/MyDrive/TPE_Mon_Travail/Déroul_Vol_Test_Nett_TF-IDF.csv"
# Chemin d'accès du dossier contenant les autres documents (déjà TF-IDF)
path_documents_folder = "/content/drive/MyDrive/TPE_Mon_Travail/BD_Fusion-TF_IDF_D-Vol1.csv"

# Charger le document de test (déjà TF-IDF)
test_data = pd.read_csv(path_test_document)

# Charger les autres documents (déjà TF-IDF)
all_documents = pd.read_csv(path_documents_folder)

# Sélectionner les vecteurs TF-IDF du document de test et des autres documents
test_vectors = test_data.iloc[:, 2:].values  # Supposons que les vecteurs TF-IDF commencent à la troisième colonne
documents_vectors = all_documents.iloc[:, 3:-1].values  # Exclure la première colonne (id) et la dernière colonne (cluster)

# Réduire les fonctionnalités de la matrice documents_vectors pour qu'elles correspondent à celles de la matrice test_vectors
num_features_test = test_vectors.shape[1]
documents_vectors = documents_vectors[:, :num_features_test]

# Calculer la similarité cosinus entre le document de test et chaque document dans le dossier
similarities = cosine_similarity(test_vectors, documents_vectors)

# Obtenir l'indice du document le plus similaire
most_similar_index = np.argmax(similarities)

# Obtenir l'ID du document le plus similaire
most_similar_document_id = all_documents.loc[most_similar_index, 'id']

print(f"Le document le plus similaire a l'ID : {most_similar_document_id}")


Le document le plus similaire a l'ID : BEA2020-0296.pdf


In [194]:
# Chemin d'accès du document de test (déjà TF-IDF)
path_test_document = "/content/drive/MyDrive/TPE_Mon_Travail/Déroul_Vol_Test_Nett_TF-IDF.csv"
# Chemin d'accès du DataFrame fusionné contenant la colonne "cluster"
path_merged_dataframe = "/content/drive/MyDrive/TPE_Mon_Travail/BD_Fusion-TF_IDF_D-Vol.csv"

# Charger le document de test (déjà TF-IDF)
test_data = pd.read_csv(path_test_document)

# Charger le DataFrame fusionné (contenant la colonne "cluster")
all_documents = pd.read_csv(path_merged_dataframe)

# ... (votre code pour calculer la similarité cosinus et trouver le document le plus similaire)

# Récupérer le cluster du document le plus similaire à partir du DataFrame fusionné
cluster_most_similar = all_documents.loc[most_similar_index, 'cluster']

print(f"Le document le plus similaire a l'ID : {most_similar_document_id}")
print(f"Le cluster du document le plus similaire est : {cluster_most_similar}")


Le document le plus similaire a l'ID : BEA2020-0296.pdf
Le cluster du document le plus similaire est : 0
